In [1]:
%config Completer.use_jedi = False
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import time

In [2]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 64
        self.epochs = 20
        self.lr = 0.02
        self.seed = 1
        self.log_interval = 1 # Log info at each batch
        self.precision_fractional = 3

args = Arguments()

_ = torch.manual_seed(args.seed)


# PySft

In [3]:
import syft as sy  # import the Pysyft library
hook = sy.TorchHook(torch)  # hook PyTorch to add extra functionalities like Federated and Encrypted Learning

# simulation functions
def connect_to_workers(n_workers):
    return [
        sy.VirtualWorker(hook, id=f"worker{i+1}")
        for i in range(n_workers)
    ]
def connect_to_crypto_provider():
    return sy.VirtualWorker(hook, id="crypto_provider")

workers = connect_to_workers(n_workers=2)
crypto_provider = connect_to_crypto_provider()

# Access and secret
The workers then split their data in batches and secret share their data between each others. 
Local worker (like us) never had access to the data.

In [4]:
# We don't use the whole dataset for efficiency purpose, but feel free to increase these numbers
n_train_items = 640#can be changed
n_test_items = 640#can be changed

def get_private_data_loaders(precision_fractional, workers, crypto_provider):
    
    def one_hot_of(index_tensor):
        """
        Transform to one hot tensor
        
        Example:
            [0, 3, 9]
            =>
            [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]
            
        """
        onehot_tensor = torch.zeros(*index_tensor.shape, 10) # 10 classes for MNIST
        onehot_tensor = onehot_tensor.scatter(1, index_tensor.view(-1, 1), 1)
        return onehot_tensor
        
    def secret_share(tensor):
        """
        Transform to fixed precision and secret share a tensor
        """
        return (
            tensor
            .fix_precision(precision_fractional=precision_fractional)
            .share(*workers, crypto_provider=crypto_provider, requires_grad=True)
        )
    
    transformation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True, transform=transformation),
        batch_size=args.batch_size
    )
    
    private_train_loader = [
        (secret_share(data), secret_share(one_hot_of(target)))
        for i, (data, target) in enumerate(train_loader)
        if i < n_train_items / args.batch_size
    ]
    
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, download=True, transform=transformation),
        batch_size=args.test_batch_size
    )
    
    private_test_loader = [
        (secret_share(data), secret_share(target.float()))
        for i, (data, target) in enumerate(test_loader)
        if i < n_test_items / args.test_batch_size
    ]
    
    return private_train_loader, private_test_loader
    
    
private_train_loader, private_test_loader = get_private_data_loaders(
    precision_fractional=args.precision_fractional,
    workers=workers,
    crypto_provider=crypto_provider
)

1.1%

100.1%

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


5.5%5%

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


180.4%

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw
Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


# Model specification


In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Training and testing functions

In [6]:
def train(args, model, private_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(private_train_loader): # <-- now it is a private dataset
        start_time = time.time()
        
        optimizer.zero_grad()
        
        output = model(data)
        
        # loss = F.nll_loss(output, target)  <-- not possible here
        batch_size = output.shape[0]
        loss = ((output - target)**2).sum().refresh()/batch_size
        
        loss.backward()
        
        optimizer.step()

        if batch_idx % args.log_interval == 0:
            loss = loss.get().float_precision()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime: {:.3f}s'.format(
                epoch, batch_idx * args.batch_size, len(private_train_loader) * args.batch_size,
                100. * batch_idx / len(private_train_loader), loss.item(), time.time() - start_time))

In [7]:
def test(args, model, private_test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in private_test_loader:
            start_time = time.time()
            
            output = model(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target.view_as(pred)).sum()

    correct = correct.get().float_precision()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct.item(), len(private_test_loader)* args.test_batch_size,
        100. * correct.item() / (len(private_test_loader) * args.test_batch_size)))

# Trainning

In [8]:
model = Net()
model = model.fix_precision().share(*workers, crypto_provider=crypto_provider, requires_grad=True)

optimizer = optim.SGD(model.parameters(), lr=args.lr)
optimizer = optimizer.fix_precision() 

for epoch in range(1, args.epochs + 1):
    train(args, model, private_train_loader, optimizer, epoch)
    test(args, model, private_test_loader)

/home/fjunyuan/.local/lib/python3.8/site-packages/syft/frameworks/torch/tensors/interpreters/additive_shared.py:122: UserWarning: Use dtype instead of field
  warnings.warn("Use dtype instead of field")


Train Epoch: 1 [0/640 (0%)]	Loss: 1.128000	Time: 7.122s
Train Epoch: 1 [64/640 (10%)]	Loss: 1.013000	Time: 8.410s
Train Epoch: 1 [128/640 (20%)]	Loss: 0.988000	Time: 8.142s
Train Epoch: 1 [192/640 (30%)]	Loss: 0.901000	Time: 7.185s
Train Epoch: 1 [256/640 (40%)]	Loss: 0.887000	Time: 6.988s
Train Epoch: 1 [320/640 (50%)]	Loss: 0.875000	Time: 7.344s
Train Epoch: 1 [384/640 (60%)]	Loss: 0.852000	Time: 7.173s
Train Epoch: 1 [448/640 (70%)]	Loss: 0.849000	Time: 7.142s
Train Epoch: 1 [512/640 (80%)]	Loss: 0.829000	Time: 8.226s
Train Epoch: 1 [576/640 (90%)]	Loss: 0.839000	Time: 8.102s

Test set: Accuracy: 233.0/640 (36%)

Train Epoch: 2 [0/640 (0%)]	Loss: 0.777000	Time: 7.340s
Train Epoch: 2 [64/640 (10%)]	Loss: 0.732000	Time: 7.606s
Train Epoch: 2 [128/640 (20%)]	Loss: 0.792000	Time: 8.453s
Train Epoch: 2 [192/640 (30%)]	Loss: 0.713000	Time: 8.054s
Train Epoch: 2 [256/640 (40%)]	Loss: 0.701000	Time: 7.574s
Train Epoch: 2 [320/640 (50%)]	Loss: 0.705000	Time: 7.435s
Train Epoch: 2 [384/640 (6

Train Epoch: 14 [64/640 (10%)]	Loss: 0.220000	Time: 6.950s
Train Epoch: 14 [128/640 (20%)]	Loss: 0.331000	Time: 6.993s
Train Epoch: 14 [192/640 (30%)]	Loss: 0.247000	Time: 7.179s
Train Epoch: 14 [256/640 (40%)]	Loss: 0.247000	Time: 7.014s
Train Epoch: 14 [320/640 (50%)]	Loss: 0.239000	Time: 7.028s
Train Epoch: 14 [384/640 (60%)]	Loss: 0.272000	Time: 6.929s
Train Epoch: 14 [448/640 (70%)]	Loss: 0.309000	Time: 6.988s
Train Epoch: 14 [512/640 (80%)]	Loss: 0.291000	Time: 7.034s
Train Epoch: 14 [576/640 (90%)]	Loss: 0.377000	Time: 7.001s

Test set: Accuracy: 504.0/640 (79%)

Train Epoch: 15 [0/640 (0%)]	Loss: 0.291000	Time: 7.039s
Train Epoch: 15 [64/640 (10%)]	Loss: 0.208000	Time: 6.992s
Train Epoch: 15 [128/640 (20%)]	Loss: 0.314000	Time: 7.036s
Train Epoch: 15 [192/640 (30%)]	Loss: 0.236000	Time: 7.072s
Train Epoch: 15 [256/640 (40%)]	Loss: 0.235000	Time: 7.077s
Train Epoch: 15 [320/640 (50%)]	Loss: 0.226000	Time: 7.032s
Train Epoch: 15 [384/640 (60%)]	Loss: 0.258000	Time: 7.343s
Train E